In [1]:
import pandas as pd
import numpy as np

In [2]:
import pvlib

from pvlib.pvsystem import PVSystem

from pvlib.location import Location

from pvlib.modelchain import ModelChain

from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

#Visualization
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #for 3D plots 
#import seaborn as sns

#sklearn
from sklearn.cluster import AgglomerativeClustering #Heirarchial Clustering
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans 
from sklearn.metrics import mean_squared_error, r2_score
#from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.model_selection import train_test_split #split data into training and test sets
from sklearn.tree import DecisionTreeClassifier #decision tree classifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from scipy.optimize import curve_fit
from scipy.cluster.hierarchy import dendrogram
from sklearn import decomposition
from sklearn import datasets
from sklearn import tree 


from scipy.stats import pearsonr


In [3]:
url = "/Users/zacblackburn/Desktop/3b51d13be28ca10bd0ea0db11a037480/Weather_2019ProjectA.csv"

In [1]:
df = pd.read_csv(url)

NameError: name 'pd' is not defined

In [5]:
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']

In [6]:
location = Location(latitude=30.61, longitude=-96.34)

system = PVSystem(surface_tilt=20, surface_azimuth=200,
                  module_parameters=sandia_module,
                  inverter_parameters=cec_inverter,
                  temperature_model_parameters=temperature_model_parameters)


mc = ModelChain(system, location)


In [7]:
print(mc)
print(df)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses
      Year  Month  Day  Hour  Minute  dhi  dni  ghi  wind_speed temp_air
0     2019      1    1     0      30    0    0    0         1.4      5.8
1     2019      1    1     1      30    0    0    0         1.6      5.8
2     2019      1    1     2      30    0    0    0         1.9      5.7
3     2019      1    1     3      30    0    0    0         2.3      5.5
4     2019      1    1     4      30    0    0    0         2.6      5.2
...    ...    ...  ...   ...     ...  ...  ...  ...         ...      ...
8755  2019     12   31    19      30    0    0    0         1.3      8.4
8756  2019     12   31    20      30    0    0    0         1.5 

In [8]:
df["time"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour", "Minute"]])
df.set_index(df["time"], inplace = True)

In [9]:
weatherd = df[['dhi', 'dni', 'ghi', 'wind_speed', 'temp_air']]

weatherd

In [10]:
weather = pd.DataFrame(
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'],
                       index=[pd.Timestamp('20190401 1230', tz='US/Central')])


In [11]:
mc.run_model(weatherd)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
mc.aoi

In [ ]:
mc.cell_temperature

In [ ]:
dc = mc.dc
plt.plot(dc)
plt.show()

In [ ]:
ac = mc.ac
plt.plot(ac)
plt.show()

In [ ]:
fig = plt.figure(figsize = [10.0, 10.0])#use large figure sizes for multiple subplots, the values are in inches.

ax1 = fig.add_subplot(221)#221: 2 - number of rows, 2- number of columns, 1- position of the subplot in the grid 
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.scatter(df['dhi'], df['temp_air'])
ax1.set_title('fig1')
ax1.set_xlabel('DHI')
ax1.set_ylabel('Temperature')

ax2.scatter(df['dni'], df['temp_air'])
ax2.set_title('dig2')
ax2.set_xlabel('dni')
ax2.set_ylabel('temperature')

ax3.scatter(df['wind_speed'], df['temp_air'])
ax3.set_title('fig3')
ax3.set_xlabel('wind_speed')
ax3.set_ylabel('temperature')

ax4.scatter(df['wind_speed'], df['temp_air'])
ax4.set_title('fig4')
ax4.set_xlabel('wind_speed')
ax4.set_ylabel('temp_air')

plt.show()


In [ ]:
fig = plt.figure(figsize = [10.0, 10.0])#

ax1 = fig.add_subplot(221)



ax1.scatter(df['ghi'], df['temp_air'])
ax1.set_title('fig5')
ax1.set_xlabel('ghi')
ax1.set_ylabel('temp_air')


In [ ]:
auto_kmeans = KMeans(n_clusters=2, init='k-means++', n_init=5, max_iter=300, 
                     tol=0.0001).fit(df[['DHI', 'Temperature']]) #n_clusters - number of clusters
print(auto_kmeans.inertia_)
auto_kmeans_centroids = pd.DataFrame(auto_kmeans.cluster_centers_, columns = ['DHI', 'Temperature']) 
plt.scatter(df['DHI'], df['Temperature'])
plt.scatter(auto_kmeans_centroids['DHI'], auto_kmeans_centroids['Temperature'])
plt.xlabel('DHI')
plt.ylabel('Temperature')
plt.show()

In [ ]:
auto_kmeans = KMeans(n_clusters=2, init='k-means++', n_init=5, max_iter=300, 
                     tol=0.0001).fit(df[['DNI', 'Temperature']])
auto_kmeans_centroids1 = pd.DataFrame(auto_kmeans.cluster_centers_, 
                                      columns = ['DNI', 'Temperature']) 
auto_kmeans_inertia1 = auto_kmeans.inertia_

In [ ]:
auto_kmeans = KMeans(n_clusters=3 , init='k-means++', n_init=5, max_iter=300, 
                     tol=0.0001).fit(df[['DNI', 'Temperature']])
auto_kmeans_centroids2 = pd.DataFrame(auto_kmeans.cluster_centers_, 
                                      columns = ['DNI', 'Temperature']) 
auto_kmeans_inertia2 = auto_kmeans.inertia_
plt.scatter(df['DNI'], df['Temperature'])
plt.scatter(auto_kmeans_centroids2['DNI'], auto_kmeans_centroids2['Temperature'])
plt.xlabel('DNI')
plt.ylabel('Temperature')
plt.show()


In [ ]:
auto_kmeans = KMeans(n_clusters=4 , init='k-means++', n_init=5, max_iter=300, 
                     tol=0.0001).fit(df[['Wind Speed', 'Temperature']])
auto_kmeans_centroids3 = pd.DataFrame(auto_kmeans.cluster_centers_, 
                                      columns = ['Wind Speed', 'Temperature']) 
auto_kmeans_inertia3 = auto_kmeans.inertia_
plt.scatter(df['Wind Speed'], df['Temperature'])
plt.scatter(auto_kmeans_centroids3['Wind Speed'], auto_kmeans_centroids3['Temperature'])
plt.xlabel('Wind Speed')
plt.ylabel('Temperature')
plt.show()

In [ ]:
auto_kmeans = KMeans(n_clusters=5, init='k-means++', n_init=5, max_iter=300, 
                     tol=0.0001).fit(df[['Wind Speed', 'GHI']])
auto_kmeans_centroids4 = pd.DataFrame(auto_kmeans.cluster_centers_, 
                                      columns = ['Wind Speed', 'GHI']) 
auto_kmeans_inertia4 = auto_kmeans.inertia_
plt.scatter(df['Wind Speed'], df['GHI'])
plt.scatter(auto_kmeans_centroids4['Wind Speed'], auto_kmeans_centroids4['GHI'])
plt.xlabel('Wind Speed')
plt.ylabel('GHI')
plt.show()


In [ ]:
auto_kmeans_inertia = pd.DataFrame({'inertia' : [auto_kmeans_inertia1 , 
                                                 auto_kmeans_inertia2 , 
                                                 auto_kmeans_inertia3 , 
                                                 auto_kmeans_inertia4] } )

In [ ]:
plt.bar(auto_kmeans_inertia.index + 2, auto_kmeans_inertia['inertia'])

In [ ]:
X = df[['DNI', 'Temperature']]
 
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
                                     
    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:

pca = PCA(n_components=2)
pca_comp = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(pca.components_)

scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)    
pca = PCA()

x_new = pca.fit_transform(X)
y = df['DNI']

def myplot(score,coeff,labels=None):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley, c = y)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()

myplot(x_new[:,0:2],np.transpose(pca.components_[0:2, :]))
plt.show()

In [ ]:
df_auto_reg = df[['DNI', 'Temperature']][df['DNI'].between(df['DNI'].quantile(0.1), df['DNI'].quantile(0.9))]
#df_auto_reg = df_auto[['DNI', 'Temperature']]
X = df_auto_reg['DNI']
y = df_auto_reg['Temperature']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

plt.scatter(X,y, color = 'mediumpurple')
plt.show()


plt.scatter(X_train, y_train)
plt.scatter(X_test, y_test)
plt.show()

In [ ]:
#considering outliers
df_auto_reg10 = df[['DHI','Temperature']]
X = df_auto_reg10['DHI']
y = df_auto_reg10['Temperature']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat10 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('MSE = %f, R-sq = %f' %tuple(reg_stat10))
plt.scatter(X_train, y_train,  color='coral')
plt.plot(X_test, y_pred, color='black', linestyle='dotted', linewidth=1)
plt.xlabel('DHI')
plt.ylabel('Temperature')
plt.show()

#considering between 5th and 95th percentile
df_auto_reg11 = df[['DHI','Temperature']][df['DHI'].between(df['DHI'].quantile(0.05), df['DHI'].quantile(0.95))]
X = df_auto_reg11['DHI']
y = df_auto_reg11['Temperature']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat11 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('MSE = %f, R-sq = %f' %tuple(reg_stat11))
plt.scatter(X_train, y_train,  color='springgreen')
plt.plot(X_test, y_pred, color='black', linestyle='dotted', linewidth=1)
plt.xlabel('DHI')
plt.ylabel('Temperature')
plt.show()

#considering between 20th and 80th percentile
df_auto_reg12 = df[['DHI','Temperature']][df['Temperature'].between(df['Temperature'].quantile(0.10), df['Temperature'].quantile(0.90))]
X = df_auto_reg12['DHI']
y = df_auto_reg12['Temperature']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat12 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('MSE = %f, R-sq = %f,' %tuple(reg_stat12))
plt.scatter(X_train, y_train,  color='skyblue')
plt.plot(X_test, y_pred, color='black', linestyle='dotted', linewidth=1)
plt.xlabel('DHI')
plt.ylabel('Temperature')
plt.show()

In [ ]:
df_auto_reg10 = df[['GHI', 'Wind Speed']][df['GHI'].between(df['GHI'].quantile(0.1), df['GHI'].quantile(0.9))]
X = df_auto_reg10['GHI']
y = df_auto_reg10['Wind Speed']  

#test size = 0.15 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat2 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('Testsize = 0.15, MSE = %f, R-sq = %f' %tuple(reg_stat2))
plt.scatter(X_test, y_test,  color='limegreen')
plt.scatter(X_test, y_pred, label = 'MSE = %f, R-sq = %f' % tuple(reg_stat2) , color='orangered')
plt.show()


#test size = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat2 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('Testsize = 0.25, MSE = %f, R-sq = %f' %tuple(reg_stat2))
plt.scatter(X_test, y_test,  color='maroon')
plt.scatter(X_test, y_pred, label = 'MSE = %f, R-sq = %f' % tuple(reg_stat2) , color='gold')
plt.show()

#test size = 0.70 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=9)
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('Coefficients: \n', reg.coef_)

reg_stat2 = [mean_squared_error(y_test, y_pred),r2_score(y_test, y_pred)] 
plt.title('Testsize = 0.70, MSE = %f, R-sq = %f' %tuple(reg_stat2))
plt.scatter(X_test, y_test,  color='orange')
plt.scatter(X_test, y_pred, label = 'MSE = %f, R-sq = %f' % tuple(reg_stat2) , color='black')
plt.show()

plt.scatter(X,y)
plt.show()
